In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Descriptors import MolLogP
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem import Descriptors
from rdkit.Chem import PandasTools
from rdkit.DataStructs import ExplicitBitVect
import sys
import multiprocessing
from standardiser import break_bonds, neutralise, rules, unsalt
from standardiser.utils import StandardiseException, sanity_check
%reload_ext autoreload
%autoreload 2
def warn(*args, **kwargs):
    pass 
import warnings
warnings.filterwarnings("ignore")
warnings.warn = warn
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import sys
from sklearn.metrics import cohen_kappa_score
import csv
from rdkit.Chem import MACCSkeys
from sklearn.model_selection import ShuffleSplit
import _pickle as cPickle
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit    
import bz2
from glob import glob
import _pickle as cPickle
import pickle
#Draw.DrawingOptions.atomLabelFontFace = "DejaVu Sans"
#Draw.DrawingOptions.atomLabelFontSize = 18
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from CDK_pywrapper import CDK
import pandas as pd
import os
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem import rdMolDescriptors

In [ ]:
# ==========================
# Load dataset
# ==========================
file_path = r"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Original_dataset_acute dermal.xlsx"
df = pd.read_excel(file_path)

# Pastikan ada kolom 'Outcome'
print("Kolom dataset:", df.columns)

# ==========================
# Train-test split (80:20, stratified by Outcome)
# ==========================
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['Outcome'],  # jaga distribusi Outcome tetap seimbang
    random_state=42
)

# ==========================
# Save hasil split di folder yang sama
# ==========================
folder = os.path.dirname(file_path)
train_file = os.path.join(folder, "Train_set_FDAMMD.xlsx")
test_file = os.path.join(folder, "Test_set_FDAMMD.xlsx")

train_df.to_excel(train_file, index=False)
test_df.to_excel(test_file, index=False)

print(f"Train set disimpan di: {train_file}")
print(f"Test set disimpan di: {test_file}")

print("Distribusi Outcome Train:")
print(train_df['Outcome'].value_counts(normalize=True))

print("Distribusi Outcome Test:")
print(test_df['Outcome'].value_counts(normalize=True))


# FINGEPRINTS COMPUTATION

In [ ]:
# Fungsi untuk menghitung Morgan Descriptors
def compute_morgan_fp(smiles, radius=2, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits))
    return [0] * n_bits  # Jika gagal, kembalikan vektor nol

# Fungsi untuk menghitung MACCS Keys
def compute_maccs_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return list(MACCSkeys.GenMACCSKeys(mol))
    return [0] * 167  # MACCS memiliki 167 bit

# Fungsi APF bit vector
def compute_apf_fp(smiles, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol, nBits=n_bits)
        return list(fp)
    return [0] * n_bits

# Daftar file input
input_files = [
    "C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Test_set_FDAMMD_balanced.xlsx",
    "C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Train_set_FDAMMD_balanced.xlsx"
]


# Proses setiap file input
for input_path in input_files:
    # Load dataset
    df = pd.read_excel(input_path)

    # Hitung fitur untuk setiap molekul
    df["Morgan_Descriptors"] = df["SMILES"].apply(compute_morgan_fp)
    df["MACCS_Descriptors"] = df["SMILES"].apply(compute_maccs_fp)
    df["APF_Descriptors"] = df["SMILES"].apply(compute_apf_fp)

    # Urutkan berdasarkan kolom Outcome
    df_sorted = df.sort_values(by="Outcome", ascending=True)

    # Tentukan output path (folder sama dengan input)
    folder = os.path.dirname(input_path)
    filename = os.path.splitext(os.path.basename(input_path))[0]
    output_path = os.path.join(folder, f"{filename}_with_fingerprints_sorted.xlsx")

    # Simpan hasil
    df_sorted.to_excel(output_path, index=False)
    print(f"Hasil untuk '{input_path}' telah disimpan ke '{output_path}'.")

print("Selesai memproses semua file.")


# RDKIT-CDK COMPUTATION

In [ ]:
# Fungsi bantu konversi SMILES ke Mol RDKit
def smiles_to_mol(smiles):
    try:
        return Chem.MolFromSmiles(smiles)
    except:
        return None

# Fungsi hitung deskriptor RDKit
def calculate_rdkit_descriptors(mol):
    columns = [
        'Molecular Weight', 'logP', 'LabuteASA', 'TPSA', 'AMW', 'NumRotatableBonds',
        'NumAromaticRings', 'NumSaturatedRings', 'NumAliphaticRings', 'NumAromaticHeterocycles',
        'NumSaturatedHeterocycles', 'NumAliphaticHeterocycles', 'NumAromaticCarbocycles',
        'NumSaturatedCarbocycles', 'NumAliphaticCarbocycles', 'FractionCSP3',
        'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v', 'Chi4v',
        'Chi1n', 'Chi2n', 'Chi3n', 'Chi4n',
        'HallKierAlpha',
        "Heavy Atom Count", "Ring Count", "Num H Donors", "Num H Acceptors"
    ]
    if mol is None:
        return {col: None for col in columns}
    return {
        'Molecular Weight': Descriptors.MolWt(mol),
        'logP': Descriptors.MolLogP(mol),
        'LabuteASA': rdMolDescriptors.CalcLabuteASA(mol),
        'TPSA': Descriptors.TPSA(mol),
        'AMW': Descriptors.MolWt(mol),
        'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
        'NumAromaticRings': Descriptors.NumAromaticRings(mol),
        'NumSaturatedRings': Descriptors.NumSaturatedRings(mol),
        'NumAliphaticRings': Descriptors.NumAliphaticRings(mol),
        'NumAromaticHeterocycles': Descriptors.NumAromaticHeterocycles(mol),
        'NumSaturatedHeterocycles': Descriptors.NumSaturatedHeterocycles(mol),
        'NumAliphaticHeterocycles': Descriptors.NumAliphaticHeterocycles(mol),
        'NumAromaticCarbocycles': Descriptors.NumAromaticCarbocycles(mol),
        'NumSaturatedCarbocycles': Descriptors.NumSaturatedCarbocycles(mol),
        'NumAliphaticCarbocycles': Descriptors.NumAliphaticCarbocycles(mol),
        'FractionCSP3': Descriptors.FractionCSP3(mol),
        'Chi0v': Descriptors.Chi0v(mol),
        'Chi1v': Descriptors.Chi1v(mol),
        'Chi2v': Descriptors.Chi2v(mol),
        'Chi3v': Descriptors.Chi3v(mol),
        'Chi4v': Descriptors.Chi4v(mol),
        'Chi1n': Descriptors.Chi1n(mol),
        'Chi2n': Descriptors.Chi2n(mol),
        'Chi3n': Descriptors.Chi3n(mol),
        'Chi4n': Descriptors.Chi4n(mol),
        'HallKierAlpha': Descriptors.HallKierAlpha(mol),
        'Heavy Atom Count': Descriptors.HeavyAtomCount(mol),
        'Ring Count': Descriptors.RingCount(mol),
        'Num H Donors': Descriptors.NumHDonors(mol),
        'Num H Acceptors': Descriptors.NumHAcceptors(mol)
    }

# Inisialisasi CDK
cdk = CDK()

# List file input
input_files = [
    "C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Train_set_FDAMMD_balanced_with_fingerprints_sorted.xlsx",
    "C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Test_set_FDAMMD_balanced_with_fingerprints_sorted.xlsx"

]

# Fitur CDK yang ingin diambil
cdk_features = [
    'ALogP', 'ALogp2', 'AMR', 'MLogP', 'nAtomP', 'naAromAtom', 'bpol',
    'nB', 'ECCEN', 'fragC', 'nHBAcc', 'nHBDon', 'nAtomLAC', 'nAtomLC',
    'PetitjeanNumber', 'nRotB', 'LipinskiFailures', 'TopoPSA', 'VAdjMat',
    'XLogP', 'Fsp3'
]

# Proses setiap file
for input_path in input_files:
    # Baca data
    df = pd.read_excel(input_path)
    
    # Buat Mol RDKit
    df['Mol'] = df['SMILES'].apply(smiles_to_mol)
    
    # Hitung deskriptor RDKit
    df['RDKit_Descriptors'] = df['Mol'].apply(calculate_rdkit_descriptors)
    
    # Ekspansi dict RDKit ke kolom
    rdkit_df = pd.json_normalize(df['RDKit_Descriptors'])
    
    # Hitung fitur CDK
    cdk_descriptors = cdk.calculate(df['Mol'])
    cdk_df = cdk_descriptors[cdk_features].reset_index(drop=True)
    
    # Gabungkan semua fitur ke DataFrame utama
    df_final = pd.concat([df.drop(columns=['Mol', 'RDKit_Descriptors']), rdkit_df, cdk_df], axis=1)
    
    # Tentukan output path
    output_path = input_path.replace('.xlsx', '_with_RDKit_and_CDK_features.xlsx')
    
    # Simpan hasil
    df_final.to_excel(output_path, index=False)
    print(f"Dataset '{input_path}' dengan RDKit + CDK features sudah disimpan di:\n{output_path}")

print("Selesai memproses semua file.")


In [ ]:
# List file input Excel
input_files = [
   "C:\Fauzan\Manuskrip QSAR 1\Major Revision\Carcinogencity (manual split)\Dataset\Train_set_Carcinogenicity_with_fingerprints_sorted_with_RDKit_and_CDK_features.xlsx", 
    "C:\Fauzan\Manuskrip QSAR 1\Major Revision\Carcinogencity (manual split)\Dataset\Test_set__Carcinogenicity_with_fingerprints_sorted_with_RDKit_and_CDK_features.xlsx"]


for file_path in input_files:
    # Baca Excel
    df = pd.read_excel(file_path)
    
    # Pastikan kolom 'Outcome' ada
    if 'Outcome' not in df.columns:
        print(f"Kolom 'Outcome' tidak ditemukan di {file_path}.")
        continue
    
    # Informasi dasar
    print(f"\nFile: {file_path}")
    print(f"Total data: {len(df)}")
    print("Distribusi Outcome:")
    print(df['Outcome'].value_counts())
    
    # Visualisasi distribusi Outcome
    plt.figure(figsize=(8, 6))
    df['Outcome'].value_counts().plot(kind='bar', color='skyblue')
    plt.title(f'Distribusi Outcome - {file_path.split("\\")[-1]}')
    plt.xlabel('Outcome')
    plt.ylabel('Frekuensi')
    plt.xticks(rotation=45)
    plt.show()


# CONVERT TO SDF (optional)

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdmolfiles
import os

# Path file Excel
file_path = r"C:\Fauzan\Additional Endpoint\Hemato Toxicity\dataset\dataset_dengan_fitur_sorted_with_RDKit_and_CDK_features.xlsx"

# Membaca file Excel
moldf = pd.read_excel(file_path)

# Pastikan kolom SMILES ada
if 'SMILES' not in moldf.columns:
    raise ValueError("Kolom 'SMILES' tidak ditemukan dalam dataset!")

# Mengonversi SMILES menjadi objek Mol
moldf['Mol'] = moldf['SMILES'].apply(Chem.MolFromSmiles)

# Mendapatkan folder dari path file input
folder_path = os.path.dirname(file_path)

# Membuat path output untuk file SDF
output_sdf_path = os.path.join(folder_path, "output_file.sdf")

# Membuat objek SDF writer
w = rdmolfiles.SDWriter(output_sdf_path)

# Menulis setiap molekul ke dalam file SDF
for _, row in moldf.iterrows():
    mol = row['Mol']
    if mol:  # Jika molekul valid
        # Menambahkan properti lain ke dalam molekul
        for col in moldf.columns:
            if col != 'Mol' and col != 'SMILES':  # Menghindari kolom 'Mol' dan 'SMILES'
                mol.SetProp(col, str(row[col]))
        # Menulis ke dalam file SDF
        w.write(mol)

# Menutup file SDF setelah selesai
w.close()

print(f"Konversi selesai! File SDF telah disimpan di: {output_sdf_path}")

# BACA FILE SDF (optional)

In [ ]:
from rdkit import Chem
import pandas as pd
import matplotlib.pyplot as plt

# Path ke file SDF
sdf_file_path = r"C:\Fauzan\Additional Endpoint\Hemato Toxicity\dataset\output_file.sdf"

# Membaca file SDF menggunakan RDKit
suppl = Chem.SDMolSupplier(sdf_file_path)

# Menyiapkan list untuk menampung data molekul dan properti
data = []
properties = set()  # Menggunakan set untuk memastikan tidak ada duplikat properti

# Mengekstrak informasi dari file SDF
for mol in suppl:
    if mol is not None:  # Pastikan molekul valid
        # Mengambil nilai 'Outcome' dan kolom lainnya
        outcome = mol.GetProp('Outcome') if mol.HasProp('Outcome') else None
        # Tambahkan data yang relevan ke dalam list
        data.append({'SMILES': Chem.MolToSmiles(mol), 'Outcome': outcome})
        
        # Ekstraksi nama properti (index) yang ada pada molekul
        for prop_name in mol.GetPropNames():
            properties.add(prop_name)

# Membuat DataFrame dari data yang diekstrak
df = pd.DataFrame(data)

# Menampilkan distribusi data pada kolom 'Outcome'
print(f"Total data: {len(df)}")
print(df['Outcome'].value_counts())

# Visualisasi distribusi Outcome
plt.figure(figsize=(8, 6))
df['Outcome'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribusi Outcome dalam Dataset')
plt.xlabel('Outcome')
plt.ylabel('Frekuensi')
plt.xticks(rotation=45)
plt.show()

# Menampilkan nama properti yang ditemukan dalam file SDF
print("Properti yang tersedia dalam file SDF:")
for prop in properties:
    print(prop)


# SPLIT DATASET (optional)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import PandasTools

# Load SDF file
sdf_file_path = r"C:\Fauzan\Additional Endpoint\Hemato Toxicity\dataset\output_file.sdf"
df = PandasTools.LoadSDF(sdf_file_path)

# Pastikan kolom Outcome bertipe int (jika belum)
df['Outcome'] = df['Outcome'].astype(int)

# Split train-test
train_df, test_df = train_test_split(df, test_size=0.1, stratify=df['Outcome'], random_state=42)

# Fungsi untuk simpan DataFrame ke SDF dan menyimpan semua properti kolom
def save_to_sdf(df, sdf_file):
    writer = Chem.SDWriter(sdf_file)
    for _, row in df.iterrows():
        mol = row['ROMol']
        if mol is None:
            continue

        # Tambahkan semua kolom lain sebagai properti
        for col in df.columns:
            if col == 'ROMol':
                continue
            val = row[col]
            # Handle NaN dan float/int ke string
            if pd.isnull(val):
                val = ""
            else:
                val = str(val)
            mol.SetProp(col, val)

        writer.write(mol)
    writer.close()

# Path simpan
train_file_path = r"C:\Fauzan\Additional Endpoint\Hemato Toxicity\dataset\train_set.sdf"
test_file_path = r"C:\Fauzan\Additional Endpoint\Hemato Toxicity\dataset\test_set.sdf"

# Simpan ke file
save_to_sdf(train_df, train_file_path)
save_to_sdf(test_df, test_file_path)

print("Train and Test sets saved successfully with all features.")


# BACA TEST TRAIN SDF

In [ ]:
from rdkit import Chem
import pandas as pd
import matplotlib.pyplot as plt

# Path ke file SDF untuk train set dan test set
train_sdf_file_path = r"C:\Fauzan\Additional Endpoint\Hemato Toxicity\dataset\train_set.sdf"
test_sdf_file_path = r"C:\Fauzan\Additional Endpoint\Hemato Toxicity\dataset\test_set.sdf"


# Fungsi untuk membaca file SDF dan mengekstrak data
def extract_data_from_sdf(sdf_file_path):
    # Membaca file SDF menggunakan RDKit
    suppl = Chem.SDMolSupplier(sdf_file_path)

    # Menyiapkan list untuk menampung data molekul dan properti
    data = []
    properties = set()  # Menggunakan set untuk memastikan tidak ada duplikat properti

    # Mengekstrak informasi dari file SDF
    for mol in suppl:
        if mol is not None:  # Pastikan molekul valid
            # Mengambil nilai 'Outcome' dan kolom lainnya
            outcome = mol.GetProp('Outcome') if mol.HasProp('Outcome') else None
            # Tambahkan data yang relevan ke dalam list
            data.append({'SMILES': Chem.MolToSmiles(mol), 'Outcome': outcome})

            # Ekstraksi nama properti (index) yang ada pada molekul
            for prop_name in mol.GetPropNames():
                properties.add(prop_name)

    # Membuat DataFrame dari data yang diekstrak
    df = pd.DataFrame(data)

    return df, properties

# Ekstrak data untuk train set
train_df, train_properties = extract_data_from_sdf(train_sdf_file_path)

# Ekstrak data untuk test set
test_df, test_properties = extract_data_from_sdf(test_sdf_file_path)

# Menampilkan distribusi data pada kolom 'Outcome' untuk train set
print(f"Total data train: {len(train_df)}")
print(train_df['Outcome'].value_counts())

# Visualisasi distribusi Outcome pada train set
plt.figure(figsize=(8, 6))
train_df['Outcome'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribusi Outcome dalam Train Set')
plt.xlabel('Outcome')
plt.ylabel('Frekuensi')
plt.xticks(rotation=45)
plt.show()

# Menampilkan distribusi data pada kolom 'Outcome' untuk test set
print(f"Total data test: {len(test_df)}")
print(test_df['Outcome'].value_counts())

# Visualisasi distribusi Outcome pada test set
plt.figure(figsize=(8, 6))
test_df['Outcome'].value_counts().plot(kind='bar', color='salmon')
plt.title('Distribusi Outcome dalam Test Set')
plt.xlabel('Outcome')
plt.ylabel('Frekuensi')
plt.xticks(rotation=45)
plt.show()

# Menampilkan nama properti yang ditemukan dalam file SDF
print("Properti yang tersedia dalam file SDF (train set):")
for prop in train_properties:
    print(prop)

print("Properti yang tersedia dalam file SDF (test set):")
for prop in test_properties:
    print(prop)